Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [220]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import numpy as np

## Build Spark Session

In [2]:
spark = SparkSession.builder.getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [6]:
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [8]:
train_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

**Show 5 rows.**

In [7]:
train_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [9]:
train_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [11]:
train_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [13]:
train_df.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [25]:
survived = train_df.select(F.col('Survived')).where(F.col('Survived') == 1).count()
not_survived = train_df.select(F.col('Survived')).where(F.col('Survived') == 0).count()
# survived_groupped_df = train_df.groupBy("Survived").count()

**Display your result:**

In [28]:
print(survived)
print(not_survived)

342
549


**Can you display your answer in ratio form?(Hint: Use UDF.)**






In [37]:
from pyspark.sql.functions import udf
@udf(returnType=DoubleType())
def ratio_of_survived(x):
    return x / train_df.count()

# getRatio = F.udf(lambda x: round(x/train_count,2), DoubleType())
# survived_groupped_df = survived_groupped_df.withColumn("Ratio", getRatio('count'))

In [39]:
survived / train_df.count()

0.3838383838383838

**Can you get the number of males and females?**


In [43]:
male = train_df.select(F.col('Sex')).where(F.col('Sex') == 'male').count()
print(male)
female = train_df.select(F.col('Sex')).where(F.col('Sex') == 'female').count()
print(female)

#gender_grouped_df=train.groupBy("Sex").count()
#gender_grouped_df.show()

577
314


**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [45]:
train_df.groupby('Sex').agg(F.mean("Survived")).show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



In [51]:
train_df.groupby('Sex').agg(F.sum("Survived")).show()

+------+-------------+
|   Sex|sum(Survived)|
+------+-------------+
|female|          233|
|  male|          109|
+------+-------------+



**Create temporary view PySpark:**

In [52]:
train_df.createOrReplaceTempView("TEMP_TABLE")

**How many people survived, and how many didn't survive? By SQL:**

In [63]:
spark.sql("select count(PassengerId) as survived from TEMP_TABLE where Survived == 1").show()
spark.sql("select count(PassengerId) as not_survived from TEMP_TABLE where Survived == 0").show()

# spark.sql("SELECT count(Survived) FROM train GROUP BY Survived").show()

+--------+
|survived|
+--------+
|     342|
+--------+

+------------+
|not_survived|
+------------+
|         549|
+------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [67]:
spark.sql("select Sex, sum(Survived) as survived from TEMP_TABLE group by Sex").show()
# spark.sql("SELECT Sex, round(SUM(Survived)/count(1),2) as ratio  FROM train GROUP BY Sex").show()

+------+--------+
|   Sex|survived|
+------+--------+
|female|     233|
|  male|     109|
+------+--------+



**Display a ratio for p-class:**


In [87]:
train_df.groupby('Pclass').count().show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [368]:
combined = train_df.union(test_df)
combined.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display count:**

In [369]:
combined.count()

1329

**Temporary view PySpark:**

In [370]:
combined.createOrReplaceTempView("NEW_TABLE")

**Can you define the number of null values in each column?**


In [371]:
combined.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in combined.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [372]:
null_df = combined.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in combined.columns])

## Preprocessing 

**Can you show me the name column from your temporary table?**

In [373]:
spark.sql("select Name from NEW_TABLE").show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [374]:
combined = combined.withColumn('Title', F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [375]:
spark.sql("select Title, count(Title) from combined group by Title").show()

+--------+------------+
|   Title|count(Title)|
+--------+------------+
|     Don|           1|
|    Miss|         257|
|Countess|           2|
|     Col|           4|
|     Rev|           9|
|    Lady|           2|
|  Master|          56|
|     Mme|           1|
|    Capt|           2|
|      Mr|         786|
|      Dr|          11|
|     Mrs|         186|
|     Sir|           2|
|Jonkheer|           2|
|    Mlle|           4|
|   Major|           3|
|      Ms|           1|
+--------+------------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [376]:
titles_map = {'Dr':"rare", 'Rev':"rare", 'Major':"rare", 'Col':"rare", 'Mlle':"rare", 'Capt':"rare", 'Don':"rare", 'Jonkheer':"rare", 'Countess':"rare", 'Ms':"rare", 'Sir':"rare", 'Lady':"rare", 'Mme':"rare"}

**Run the function:**

In [377]:
def impute_title(title):
    return titles_map[title]

**Apply the function on "Title" column using UDF:**

In [378]:
impute_titleUDF = udf(lambda z:impute_title(z) if z in titles_map else z)   
combined = combined.withColumn("Title", impute_titleUDF(F.col("Title")))

**Display "Title" from table and group by "Title" column:**

In [379]:
combined.groupby('Title').agg(F.count("Title")).show()

+------+------------+
| Title|count(Title)|
+------+------------+
|  rare|          44|
|  Miss|         257|
|Master|          56|
|    Mr|         786|
|   Mrs|         186|
+------+------------+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [380]:
age_avg = combined.agg(F.mean("Age")).collect()[0][0]
# round(spark.sql("SELECT AVG(Age) FROM combined").collect()[0][0])
age_avg

30.079501879699244

**Fill missing age with age mean:**

In [381]:
combined = combined.na.fill(age_avg, subset=['Age'])

In [382]:
combined.select([F.count(F.when(F.isnull('Age'), 'Age'))]).show()

+-------------------------------------------+
|count(CASE WHEN (Age IS NULL) THEN Age END)|
+-------------------------------------------+
|                                          0|
+-------------------------------------------+



## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [383]:
grouped_Embarked = combined.select('Embarked').groupby('Embarked').agg(F.count("Embarked").alias("count")).orderBy('count', ascending=False)

**Show groupped_Embarked:**

In [384]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    0|
+--------+-----+



**Get the groupped_Embarked:** 

In [385]:
Embarked_mode = grouped_Embarked.collect()[0][0]
Embarked_mode

'S'

In [386]:
combined.select([F.count(F.when(F.isnull('Embarked'), 'Embarked'))]).show()

+-----------------------------------------------------+
|count(CASE WHEN (Embarked IS NULL) THEN Embarked END)|
+-----------------------------------------------------+
|                                                    3|
+-----------------------------------------------------+



**Fill missing values with grouped_Embarked:**

In [387]:
combined = combined.na.fill(Embarked_mode, subset=['Embarked'])

In [388]:
combined.select([F.count(F.when(F.isnull('Embarked'), 'Embarked'))]).show()

+-----------------------------------------------------+
|count(CASE WHEN (Embarked IS NULL) THEN Embarked END)|
+-----------------------------------------------------+
|                                                    0|
+-----------------------------------------------------+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



**Fill missing values with "U":**

In [389]:
combined = combined.withColumn('Cabin', F.when(combined.Cabin.isNull(), 
F.lit("U")).otherwise(combined.Cabin.substr(1,1)))

**Show the result:**

In [390]:
combined.select('Cabin').show()

+-----+
|Cabin|
+-----+
|    U|
|    C|
|    U|
|    C|
|    U|
|    U|
|    E|
|    U|
|    U|
|    U|
|    G|
|    C|
|    U|
|    U|
|    U|
|    U|
|    U|
|    U|
|    U|
|    U|
+-----+
only showing top 20 rows



**Create the temporary view:**

In [391]:
combined.createOrReplaceTempView('combined')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [392]:
grouped_Cabin = combined.select('Cabin').groupby('Cabin').agg(F.count("Cabin").alias("count")).orderBy('count', ascending=False)
grouped_Cabin.show()

+-----+-----+
|Cabin|count|
+-----+-----+
|    U| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**Use Pipline to fit and transform:**

In [440]:
combined = combined.drop('Name', 'PassengerId')

In [441]:
categoricalCols = [field for (field, dataType) in combined.dtypes if dataType == "string"]
categoricalCols

['Sex', 'Ticket', 'Cabin', 'Embarked', 'Title']

In [442]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Sex_Index', 'Ticket_Index', 'Cabin_Index', 'Embarked_Index', 'Title_Index']

In [443]:
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

In [444]:
numericCols = [field for (field,dataType) in combined.dtypes
              if (((dataType=='double') | (dataType=='int')) & (field!='Survived') )]
numericCols

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [445]:
assemblerInputs = indexOutputCols + numericCols
assemblerInputs

['Sex_Index',
 'Ticket_Index',
 'Cabin_Index',
 'Embarked_Index',
 'Title_Index',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

In [465]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived', maxBins=620, seed=42)

In [466]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

In [467]:
from pyspark.ml import Pipeline
pipeline =Pipeline(stages = [stringIndexer,vecAssembler,rf])

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [468]:
train, test = combined.randomSplit([0.8, 0.2], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 1071
Test Dataset Count: 258


**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [469]:
pipelineModel = pipeline.fit(train)

In [470]:
predDF = pipelineModel.transform(test)

In [471]:
predDF.select('features','Survived','prediction').show(5)

+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|[1.0,19.0,1.0,0.0...|       0|       1.0|
|[0.0,276.0,2.0,1....|       0|       0.0|
|[0.0,573.0,0.0,1....|       0|       1.0|
|(10,[1,2,5,6,9],[...|       0|       0.0|
|[0.0,304.0,0.0,1....|       0|       0.0|
+--------------------+--------+----------+
only showing top 5 rows



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [472]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
accuracy = MulticlassClassificationEvaluator(predictionCol='prediction',
                                         labelCol='Survived',
                                         metricName='accuracy').evaluate(predDF)
print(f"the accuracy is {accuracy}")

the accuracy is 0.9095744680851063


**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
